In [48]:
from datetime import datetime
import random
import string
from typing import Optional

import pandas as pd

from django.contrib.auth.hashers import PBKDF2PasswordHasher

In [49]:
LETTERS = string.ascii_letters
NUMBERS = string.digits


def random_string(length: int = 20):
    """
    Generates a random password having the specified length
    :length -> length of password to be generated. Defaults to 8
        if nothing is specified.
    :returns string <class 'str'>
    """
    # create alphanumerical from string constants
    printable = f"{LETTERS}{NUMBERS}"

    # convert printable from string to list and shuffle
    printable = list(printable)
    random.shuffle(printable)

    # generate random password and convert to string
    random_password = random.choices(printable, k=length)
    random_password = "".join(random_password)
    return random_password

In [50]:
registration = pd.read_csv("./data/registration.csv")


def sanitize_team_name(name: str):
    special = "@.+-_"
    team_name = name.strip()
    team_name = team_name.replace(" ", "_")

    team_name = "".join(c for c in team_name if c.isalnum() or c in special)

    return team_name


registration["date"] = pd.to_datetime(
    registration["Časová značka"], format="%d.%m.%Y %H:%M:%S"
)
registration["team_name"] = registration["Název týmu"].apply(sanitize_team_name)
registration["email"] = registration["Kontaktní email Kapitána (školní)"].apply(
    lambda x: x.strip()
)
registration["first_name"] = (
    registration["Jméno a příjmení Kapitána"].str.strip().str.split(" ").str[0]
)
registration["last_name"] = (
    registration["Jméno a příjmení Kapitána"].str.strip().str.split(" ").str[-1]
)

registration["password"] = [random_string(12) for i in range(len(registration))]

In [51]:
# display(registration[["date", "team_name", "email", "first_name", "last_name"]])
registration[
    ["date", "team_name", "email", "first_name", "last_name", "password"]
].to_csv("./data/users.csv")

In [52]:
users = pd.read_csv("./data/users.csv")

In [53]:
def get_password_hash(
    password: str, salt: Optional[str] = None, iterations: int = 36000
):
    hasher = PBKDF2PasswordHasher()
    if salt is None:
        salt = random_string(12)
    return hasher.encode(password, salt, iterations)

In [55]:
contest_id = 1

template = string.Template(
    """
    {
        "fields": {
            "real_start": "${date_joined}",
            "contest_id": ${contest_id},
            "user_id": ${user_index},
            "virtual": 0,
            "score": 0,
            "is_disqualified": 0
        },
        "model": "judge.contestparticipation",
        "pk": ${user_index}
    },
    {
        "fields": {
            "contest_id": ${contest_id},
            "profile_id": ${user_index}
        },
        "model": "judge.contest_view_contest_scoreboard",
        "pk": ${user_index}
    },
    {
        "fields": {
            "about": "",
            "ace_theme": "github",
            "current_contest": null,
            "display_rank": "user",
            "ip": "",
            "language": 1,
            "last_access": "${date_joined}",
            "math_engine": "auto",
            "mute": false,
            "organizations": [
                1
            ],
            "performance_points": 0.0,
            "points": 0.0,
            "problem_count": 0,
            "rating": null,
            "timezone": "Europe/Prague",
            "user": ${user_index},
            "user_script": "",
            "current_contest_id": ${user_index}
        },
        "model": "judge.profile",
        "pk": ${user_index}
    },
    {
        "fields": {
            "date_joined": "${date_joined}",
            "email": "${email}",
            "first_name": "${first_name}",
            "last_name": "${last_name}",
            "groups": [

            ],
            "is_active": true,
            "is_staff": false,
            "is_superuser": false,
            "last_login": "${date_joined}",
            "password": "${password_hash}",
            "user_permissions": [

            ],
            "username": "${username}"
        },
        "model": "auth.user",
        "pk": ${user_index}
    }"""
)

user_fixtures = []

for i, row in users.iterrows():
    user_fixtures.append(
        template.substitute(
            contest_id=contest_id,
            user_index=(11 + i),
            username=row["team_name"],
            first_name=row["first_name"],
            last_name=row["last_name"],
            password_hash=get_password_hash(row["password"]),
            date_joined=pd.to_datetime(row["date"]).isoformat(),
            email=row["email"],
        )
    )

with open("../repo/judge/fixtures/users.json", "w+") as f:
    f.write("[" + ",".join(user_fixtures) + "]")